In [2]:
import pandas as pd
import numpy as np
import folium
import math
import datetime as dt
import glob
from ipywidgets import interact, widgets
import geopandas as gpd
from shapely.geometry import Point
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import matplotlib.dates as mdate
import seaborn as sbn
import scipy.stats as st
pd.set_option('precision', 1) #be careful because this also applies to lat long
# global settings for the charts
plt.rcParams['font.family'] = 'FreeSerif'
#plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 22
plt.rcParams['figure.titlesize'] = 26
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['axes.grid.axis'] = 'y'
plt.rcParams['axes.grid'] = True
plt.rcParams['legend.fontsize'] = 18

In [20]:
acidentes = pd.read_csv('../radartona/incidentes/acidentes-por-radar.csv')
acidentes.n_acidentes100 = acidentes.n_acidentes100.astype(int)
acidentes.n_acidentes250 = acidentes.n_acidentes250.astype(int)
acidentes.n_acidentes500 = acidentes.n_acidentes500.astype(int)
acidentes.n_acidentes1000 = acidentes.n_acidentes1000.astype(int)
acidentes.n_acidentes2000 = acidentes.n_acidentes2000.astype(int)
acidentes.drop(acidentes.columns[0],axis=1,inplace=True)
acidentes.set_index('id_radar',inplace=True)
acidentes.head()
acidentes.append(acidentes.sum().rename('total'))#.assign(total_radar=lambda x: x.sum(1))
#len(acidentes)

,n_acidentes100,n_acidentes250,n_acidentes500,n_acidentes1000,n_acidentes2000
id_radar,,,,,
16,1,1,1,3,8
78,1,3,4,6,19
81,1,1,4,4,18
117,1,1,2,5,21
144,1,1,2,9,34
157,1,1,1,3,15
185,1,1,2,6,18
190,1,2,3,13,27
191,1,2,3,13,27
